In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import LabelEncoder

In [ ]:
import os

def open_files_in_dir(path):
    files = {}
    with os.scandir(path) as entries:
        for entry in entries:
            if entry.is_file():
                files[entry.name.split('.')[0]] = pd.read_csv(entry)
    return files

basepath = 'C:/datasets/sales/'
files_sales = open_files_in_dir(basepath)
print(files_sales.keys())

In [ ]:
files_sales['sales_train']

## DQC class

At first, we can create class which will encapsulate some information about given files.

In [ ]:
class DQC:

    def __init__(this, our_files):
        this.files = our_files

    def print_head(this, table_name):
        print(this.files[table_name].head())

    def are_there_any_duplicates(this):

        sum_of_dupl = 0
        tables = []
        for file in this.files:
            if this.files[file].duplicated().sum()!=0:
                tables.append(file)
                sum_of_dupl += this.files[file].duplicated().sum()

            else: sum_of_dupl += this.files[file].duplicated().sum()

        return f"The amount of duplicated data in all the tables: {sum_of_dupl}; Tables with duplicates: {tables}"

    def are_there_any_null_values(this):

        info = []

        for file in this.files:
            info.append(f"Table {file} contains {this.files[file].isna().sum().sum()} empty raws")

        return info

    def are_there_any_outliers(this):

        fig, ax = plt.subplots(1, 2, figsize=(10, 6))
        # add padding between the subplots
        plt.subplots_adjust(wspace=0.5)

        # draw boxplot for age in the 1st subplot
        sns.boxplot(data=this.files['sales_train']['item_price'], ax=ax[0],)
        ax[0].set_xlabel('item_price')

        sns.boxplot(data=this.files['sales_train']['item_cnt_day'], ax=ax[1],)
        ax[1].set_xlabel('item_cnt_day')

    def columns_type_and_structure(this):

        for file in this.files:
            print(f'Table "{file}" with {this.files[file].shape[0]} values:')
            print(this.files[file].dtypes)
            print('\n')

    def are_there_any_negative_values(this):

        temp_table = this.files['sales_train']
        for col in temp_table.columns:
            if temp_table[col].dtype != "O":
                negs =  len(temp_table[temp_table[col]<0])
                print(f"The precent of negative values in sales_train table in {col} column: {round(negs*100/len(temp_table),5)}")
                if negs!=0:
                    print('\n')
                    print(temp_table[temp_table[col]<0].head())
                    print('\n')

    def the_returned(this):

        sales_train = this.files['sales_train']
        returned = sales_train[sales_train['item_cnt_day']<=0]

        return returned

    def is_actually_a_return(this):

        returned = this.the_returned()
        colors = sns.color_palette('pastel')[0:5]

        plt.figure()
        plt.title('Item ids:')
        plt.pie(returned['item_id'].value_counts().head(), labels = returned['item_id'].value_counts().head().index, colors = colors)

        plt.figure()
        plt.title('Shop ids:')
        plt.pie(returned['shop_id'].value_counts().head(), labels = returned['shop_id'].value_counts().head().index, colors = colors)

        dif_date = returned['date'].nunique()
        print('Amount of unique dates:',dif_date)


In [ ]:
from itertools import combinations

def inspect_table(data):
    report = []

    for t_name, df in data.items():
        nan_report = df.isna().sum(0) / len(df)
        unique_report = df.nunique()
        types = df.infer_objects().dtypes.astype(str)
        total = [len(df) for _ in range(len(df.columns))]

        report.append( pd.DataFrame(
            data=[
                nan_report.values,
                unique_report.values,
                types,
                total
            ],
            index=['nan_report', 'unique', 'dtype', 'total'],
            columns= pd.MultiIndex.from_tuples([(col, t_name) for col in df.columns], names=["column_name", "table_name"])
        ))

    return pd.concat(report, axis=1).transpose().reset_index().sort_values('column_name').set_index(["column_name", "table_name"])

def dqc_inspection(data, report):
    report = report.reset_index()
    dqc_report = {}

    # Step: schema key consistency
    col_cnt = report.groupby('column_name').table_name.count().reset_index()
    merging_cols = list(col_cnt[col_cnt.table_name > 1].column_name)
    merging_cols__tables = report[report.column_name.isin(merging_cols)]

    consistency_table__data = {}

    for col in merging_cols:
        table_names = merging_cols__tables[merging_cols__tables.column_name == col]

        id_table_pairs = list(combinations(table_names.table_name.values, 2))

        for l_table, r_table in id_table_pairs:

            cmp_key = f"[{col}] {l_table} - {r_table}"
            l_unique = set(data[l_table][col].unique())
            r_unique = set(data[r_table][col].unique())

            inter_len = len(l_unique.intersection(r_unique))
            consistency_table__data__row = (len(l_unique) - inter_len, inter_len, len(r_unique) - inter_len)

            consistency_table__data[cmp_key] = consistency_table__data__row

    dqc_report["consistency_table"] = pd.DataFrame(data=consistency_table__data.values(), columns=['left_cnt_only', 'intersect_len', 'right_cnt_only'], index=consistency_table__data.keys())

    return dqc_report

## DQC methods

In [ ]:
dqc = DQC(files_sales)

In [ ]:
dqc.columns_type_and_structure()

6 tables, 2 of them contain test values.

There are 2935849 values in train set and 214200 in test set (about 14:1).

In test values we have amount of sold items in a month while in training set we use daily measure. Date is an object, can be converted in datetime type, then we also can lessen periods of time from date to look at some dynamics.

In [ ]:
#####added######
report = inspect_table(files_sales)
dqc_base_report = dqc_inspection(files_sales, report)
dqc_base_report['consistency_table']

In test set we have shops and items that will be unfamiliar to the future model since there is no such shops in items in the training set.

In [ ]:
dqc.print_head('sales_train')

From first 5 rows we already see that there are returns (neg item_cnt_day). We can look at it closely.

In [ ]:
dqc.are_there_any_negative_values()

Negative values can be deleted.

In [ ]:
dqc.is_actually_a_return()

Returns happen with a lot of different type of items in a lot of different shops. The dates are also pretty diverse.

In [ ]:
dqc.are_there_any_null_values()

No missing data.

In [ ]:
dqc.are_there_any_duplicates()

Found some duplicated rows. Only 6, so it can be safely deleted or ignored.

In [ ]:
dqc.are_there_any_outliers()

Explicit outliers in item_price and item_cnt_day. We can look at them closely.

In [ ]:
files_sales['sales_train'][files_sales['sales_train']['item_price']>=300000]

In [ ]:
files_sales['shops'][files_sales['shops']['shop_id']==12]

In [ ]:
files_sales['items'][(files_sales['items']['item_id']==11373)|(files_sales['items']['item_id']==20949)|(files_sales['items']['item_id']==6066)]

522 licenses in the set explains the standout price of the item. The same can be said about the amount of sold items per day: regular packing bags and delivery service.

We also see that people tend to buy things in greater numbers through online stores.


### Merging

In [ ]:
train = pd.merge(files_sales['sales_train'], files_sales['shops'], on='shop_id', how = 'left')
train = pd.merge(train, files_sales['items'], on='item_id', how = 'left')
train = pd.merge(train, files_sales['item_categories'], on='item_category_id', how = 'left')

test = pd.merge(files_sales['test'], files_sales['shops'], on='shop_id', how = 'left')
test = pd.merge(test, files_sales['items'], on='item_id', how = 'left')
test = pd.merge(test, files_sales['item_categories'], on='item_category_id', how = 'left')
test.insert(loc=0,column='date_block_num',value=train['date_block_num'].max()+1)

## ETL

In [ ]:
def length(data):

    data_len = data.shape[0]

    print(f"The amount of rows in data set: {data_len}")
    print('The amount of null values in train:',data.isna().sum().sum())

def delete_neg_in_price_and_sales(data):

    data = data[data['item_price']>0]
    data = data[data['item_cnt_day']>0]

    return data

def delete_duplicates(data):

    return data.drop_duplicates()

def date_format(data, col):

    data[col] = pd.to_datetime(data[col], format='%d.%m.%Y')

    data['year'] = data[col].dt.year
    data['month'] = data[col].dt.month
    data['day'] = data[col].dt.day

    return data

def to_file(data):

    data.to_csv('C:/datasets/sales/after_etl.csv')

    ###Only after date converting
    ## через get_dummies слишком большие размерности выходят, какой-то другой метод брать или оставлять пока?
def cat_to_num(data, col):
    cat = [col for col in data.columns if data[col].dtype=='O']
    data
    return pd.get_dummies(data, columns = col)

In [ ]:
train = delete_neg_in_price_and_sales(train)
train = date_format(train, 'date')
train['year-month'] = train['date'].dt.to_period('M')
#train = train[train['item_price']<300000]
#train = train[train['item_cnt_day']<800]
train = delete_duplicates(train)
train.head()

In [ ]:
train['shop_name'].unique()

In [ ]:
train.loc[train['shop_name']=='!Якутск Орджоникидзе, 56 фран','shop_name'] = 'Якутск Орджоникидзе, 56'
train.loc[train['shop_name']=='!Якутск ТЦ "Центральный" фран','shop_name'] = 'Якутск ТЦ "Центральный"'
train.loc[train['shop_name']=='Москва ТК "Буденовский" (пав.К7)','shop_name'] = 'Москва ТК "Буденовский"'
train.loc[train['shop_name']=='Москва ТК "Буденовский" (пав.А2)','shop_name'] = 'Москва ТК "Буденовский"'

In [ ]:
test.loc[test['shop_name']=='!Якутск Орджоникидзе, 56 фран','shop_name'] = 'Якутск Орджоникидзе, 56'
test.loc[test['shop_name']=='!Якутск ТЦ "Центральный" фран','shop_name'] = 'Якутск ТЦ "Центральный"'
test.loc[test['shop_name']=='Москва ТК "Буденовский" (пав.К7)','shop_name'] = 'Москва ТК "Буденовский"'
test.loc[test['shop_name']=='Москва ТК "Буденовский" (пав.А2)','shop_name'] = 'Москва ТК "Буденовский"'

## EDA

We should have answers to the following questions:
- What I'm working with ?
- What are the main patterns and tendencies with my data ?
- How do my data entities interact ?
- What should I do with my data to obtain good features ?
- What are the problems with the probable prediction ?
- How could I manually make predict based on data ?

In [ ]:
train.describe().T

In [ ]:
#general dynamics
plt.title('General dynamics of amount of sold items per time')
time_series=train.groupby(["date_block_num"])["item_cnt_day"].sum()
time_series.plot();

We clearly see that the series is not stationary. It has a downtrend and apparently some seasonality, around a year.

In [ ]:
date_org = train.groupby(["year-month","shop_name"])\
    ["date","item_cnt_day"].agg({"item_cnt_day":"sum"}).reset_index()
date_org.columns = ['year-month','shop_name','item_cnt_month']
top = date_org.sort_values(by='item_cnt_month', ascending=False).head(10)
sns.barplot(data=top, x="item_cnt_month", y="year-month", palette='pastel')

More often people were buying in 2013, probably due to lower prices. There is also a pattern: in winter, at the end and at the beginning of the year, sales are usually higher.

In [ ]:
#####added######
plt.title('General dynamics of mean item price per time')
time_series=train.groupby(["date_block_num"])["item_price"].mean()
time_series.plot();

In [ ]:
#####added######
t = train[train['item_price']<300000]
t = t[t['item_cnt_day']<800]
plt.title('General dependency between amount of sold items and price')
plt.ylabel('item_cnt')
plt.xlabel('item_price')
plt.scatter(t['item_price'], t['item_cnt_day'], alpha=0.5)

As we see the lesser the prices the higher the sales.

In [ ]:
#####added######
shops_look_up = train.groupby(['shop_name','date_block_num','item_name']).agg({'item_cnt_day': 'sum', 'item_price':'mean'}).reset_index()
shops_look_up['income'] = round(shops_look_up['item_cnt_day']*shops_look_up['item_price'],2)
shops_look_up = shops_look_up.groupby(['shop_name','date_block_num',]).agg({'item_cnt_day': 'sum', 'item_price':'sum','income':'sum'}).reset_index()

In [ ]:
shops_look_up.groupby('shop_name').agg({'item_cnt_day': ['mean','std', 'sum'],'date_block_num':'count'})

Based on deviation we can tell that 2 shops(Жуковский ул.Чкалова 39м² and Новосибирск ТРЦ "Галерея Новосибирск") were active only for a month. The lower std compared to mean the more homogeneous the sales are.

In [ ]:
#####added######
shops_list = shops_look_up['shop_name'].unique()
fig, axs = plt.subplots(2, 2, figsize=(25, 15))

sns.lineplot(data=shops_look_up[(28<=shops_look_up['date_block_num'])&(shops_look_up['shop_name'].isin(shops_list[:15]))], x="date_block_num", y="item_cnt_day", hue="shop_name", ax=axs[0][0])
sns.lineplot(data=shops_look_up[(28<=shops_look_up['date_block_num'])&(shops_look_up['shop_name'].isin(shops_list[15:31]))], x="date_block_num", y="item_cnt_day", hue="shop_name",ax=axs[0][1])
sns.lineplot(data=shops_look_up[(28<=shops_look_up['date_block_num'])&(shops_look_up['shop_name'].isin(shops_list[31:46]))], x="date_block_num", y="item_cnt_day", hue="shop_name",ax=axs[1][0])
sns.lineplot(data=shops_look_up[(28<=shops_look_up['date_block_num'])&(shops_look_up['shop_name'].isin(shops_list[46:]))], x="date_block_num", y="item_cnt_day", hue="shop_name",ax=axs[1][1])

In [ ]:
shops_list = shops_look_up['shop_name'].unique()
fig, axs = plt.subplots(2, 2, figsize=(25, 15))

sns.lineplot(data=shops_look_up[(28<=shops_look_up['date_block_num'])&(shops_look_up['shop_name'].isin(shops_list[:15]))], x="date_block_num", y="income", hue="shop_name", ax=axs[0][0])
sns.lineplot(data=shops_look_up[(28<=shops_look_up['date_block_num'])&(shops_look_up['shop_name'].isin(shops_list[15:31]))], x="date_block_num", y="income", hue="shop_name",ax=axs[0][1])
sns.lineplot(data=shops_look_up[(28<=shops_look_up['date_block_num'])&(shops_look_up['shop_name'].isin(shops_list[31:46]))], x="date_block_num", y="income", hue="shop_name",ax=axs[1][0])
sns.lineplot(data=shops_look_up[(28<=shops_look_up['date_block_num'])&(shops_look_up['shop_name'].isin(shops_list[46:]))], x="date_block_num", y="income", hue="shop_name",ax=axs[1][1])

While looking at the last 6 month dynamics in every available shop we see that most shops are pretty stable with Москва ТЦ "Семеновский" being the leader. Online shops tend to have high peaks in block 32. 2 shops were also closed during this period of time.

In [ ]:
train_eda = train.groupby(['year','month']).agg({'item_price': 'mean','item_name': 'count',}).reset_index()
for idx, col in enumerate(train_eda.columns[2:]):
    plt.figure()
    sns.lineplot(data=train_eda, x='month', y=col, hue='year').set_title(f"Dynamics for {col}")

As expected, prices per item in 2015 is way higher than the were 1-2 years ago. For some reason every august the also tend to get bigger. We have reverse situation in amount of sold items.

In [ ]:
import calendar


p_year_patterns = train.groupby(['day', 'month', 'year']).agg({
    'item_name': 'count',
}).reset_index()

fig, axs = plt.subplots(3, 4, figsize=(25, 15))

for month in range(1,13):
    sns.lineplot(data=p_year_patterns[p_year_patterns.month == month], x='day', y='item_name', hue='year', ax=axs[(month - 1) // 4][(month - 1) % 4]).set_title(f"{col} - dynamic in {calendar.month_name[month]}")

In [ ]:
#for numerical feathures
tr = train.copy()
tr.drop(['shop_id','item_id','item_category_id'],inplace=True, axis=1)

corr_matrix = tr.corr(method='spearman')
sns.heatmap(corr_matrix, annot=True);

There is a correlation between the values obtained from the date. Before modeling some of it will be removed.

The biggest correlation target formed with item_price.

### Categorial feathures

In [ ]:
train['shop_name'].value_counts().head(20)

As we can see names have a pattern: location - type - shop name. We can extract some information here, maybe it will be useful.

In [ ]:
train['location'] = train['shop_name'].str.split(" ").str[0]
train['shop_type'] = train['shop_name'].str.split(" ").str[1]

In [ ]:
test['location'] = test['shop_name'].str.split(" ").str[0]
test['shop_type'] = test['shop_name'].str.split(" ").str[1]

In [ ]:
train['location'].value_counts()
train['shop_type'].value_counts()

In [ ]:
train.loc[train['location']=='Цифровой','location'] = 'Цифровой склад'
train.loc[train['location']=='Сергиев','location'] = 'Сергиев посад'
train.loc[train['location']=='!Якутск','location'] = 'Якутск'
train.loc[train['shop_type']=='Орджоникидзе,','shop_type'] = 'Не указан'
train.loc[train['shop_type']=='Посад','shop_type'] = 'ТЦ'
train.loc[train['shop_type']=='"Распродажа"','shop_type'] = 'Не указан'
train.loc[train['shop_type']=='(Плехановская,','shop_type'] = 'Не указан'
train.loc[train['shop_type']=='склад','shop_type'] = 'Не указан'
train.loc[train['shop_type']=='ул.','shop_type'] = 'Не указан'
train.loc[train['shop_type']=='МТРЦ','shop_type'] = 'ТРЦ'
train.loc[train['shop_type']=='Торговля','shop_type'] = 'Не указан'

In [ ]:
test.loc[test['location']=='Цифровой','location'] = 'Цифровой склад'
test.loc[test['location']=='Сергиев','location'] = 'Сергиев посад'
test.loc[test['location']=='!Якутск','location'] = 'Якутск'
test.loc[test['shop_type']=='Орджоникидзе,','shop_type'] = 'Не указан'
test.loc[test['shop_type']=='Посад','shop_type'] = 'ТЦ'
test.loc[test['shop_type']=='"Распродажа"','shop_type'] = 'Не указан'
test.loc[test['shop_type']=='(Плехановская,','shop_type'] = 'Не указан'
test.loc[test['shop_type']=='склад','shop_type'] = 'Не указан'
test.loc[test['shop_type']=='ул.','shop_type'] = 'Не указан'
test.loc[test['shop_type']=='МТРЦ','shop_type'] = 'ТРЦ'
test.loc[test['shop_type']=='Торговля','shop_type'] = 'Не указан'

In the names we also can see some random symbols, "!" for example.

In [ ]:
train[train['shop_name'].str.contains('Якутск')==True]['shop_name'].unique()

In [ ]:
train['shop_name'].unique()

In [ ]:
train['item_category'] = train['item_category_name'].str.split(" - ").str[0]

In [ ]:
test['item_category'] = test['item_category_name'].str.split(" - ").str[0]
test["subcat"] = test["item_category_name"].str.split(" - ").map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())

In [ ]:
train["subcat"] = train["item_category_name"].str.split(" - ").map(
    lambda x: x[1].strip() if len(x) > 1 else x[0].strip()
)

In [ ]:
train.head()

In [ ]:
for col in ['location','shop_type','shop_name', 'item_category_name']:
    top_10_locs = train[col].value_counts().head(10)
    top_10_locs = top_10_locs.to_frame().reset_index()
    top_10_locs.columns = [col, 'count']
    sns.barplot(data=top_10_locs, x='count', y=col, palette = 'pastel')
    plt.title(f'Top 10 of {col}')
    plt.ylabel(col)
    plt.xlabel('Count')
    plt.show()

Moscow and it's shops are apperently the most common ones. ТЦ are quite popular.

### What can be done with data

- First of all, we can make data take form that will be more convinient for our main goal. It means that info can be grouped by shops and items, than for every group we can calculate the amount of soled products. Due to the fact that price of an item is not fixed we can try to take mean value.

- As it was stated earlier, seasonality and trend should be looked into.

- While working with time series, it is a common practice to create "lagged" copies of the series. Lagging a time series means to shift its values forward one or more time steps, or equivalently, to shift the times in its index backward one or more steps. We can also do that.